# Creacion Consolidado por Materia Penal

Comenzaremos el analisis de la data consolidando la info en un unico DataSet, para ello vamos a considerar los siguientes puntos:


- DATA:  Se considerará toda las causas que ingresaron al Sistema desde 2015 al 2019, esto ya que se cuenta además con la data de las audiencias en ese período.


- No se consideran causas anteriores a este período.

- Se considerarán Causas sin termino.

In [2]:
import pandas as pd
import numpy as np
import os
import warnings

from tqdm import tqdm
from unicodedata import normalize
from src.data import cleandata
from src.data import transformdata

In [4]:
warnings.filterwarnings(action="ignore")
tqdm.pandas()

In [3]:
pd.set_option('display.max_columns', 100)

In [4]:
path_processed = "../data/processed/pjud"

#### Paso 1 : Consolidacion de Dataset x Materia y x rol

In [5]:
df_ingresos_materia = pd.read_feather(f"{path_processed}/IngresosMateria.feather")
print(f"{len(df_ingresos_materia)} causas en el dataset de ingresos por materia")

3210643 causas en el dataset de ingresos por materia


In [6]:
df_termino_materia = pd.read_feather(f"{path_processed}/TerminoMateria.feather")
print(f"Existen : {len(df_termino_materia)} causas en el dataset de termino por materia")

Existen : 3143099 causas en el dataset de termino por materia


Analizamos la data, uniendo ambos dataset.

In [7]:
df_fulldata_materia = pd.merge(df_ingresos_materia, df_termino_materia, how='outer', on=['COD. TRIBUNAL','RIT','COD. MATERIA'])
print(f"Existen: {len(df_fulldata_materia)} causas en total uniendo ambos dataset, aca existen datos sin registro de ingreso")

Existen: 4249511 causas en total uniendo ambos dataset, aca existen datos sin registro de ingreso


### Analisis de la Data Causas por Materia

De estos dos dataset debo considerar:

- Caso 1: el dataset df_causas_terminadas completo
- Caso 2: Debo agregar además las causas que tengan Fecha Ingreso_x pero que aún no terminan.
- Caso 3: Debo analizar las causas que no tienen Fecha Ingreso_x pero si tienen Fecha Ingreso_y y Fecha Ingreso_y, ya que deben ser las causas que no fueron tomadas por la consulta de Ingresos. Acá se debe reemplazar los datos obtenidos en Termino Materia a Ingresos.

Luego de tener estos tres dataset debo concatenarlos

In [8]:
df_fulldata_materia

,level_0_x,index_x,COD. CORTE_x,CORTE_x,COD. TRIBUNAL,TRIBUNAL_x,RIT,TIPO CAUSA_x,COD. MATERIA,MATERIA_x,FECHA INGRESO_x,MES INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,level_0_y,index_y,COD. CORTE_y,CORTE_y,TRIBUNAL_y,TIPO CAUSA_y,MATERIA_y,FECHA INGRESO_y,FECHA TERMINO,MES TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
0,0.0,0.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2018,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,abr-18,2018.0,1.0,0.0,0.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,2018-06-01,jun-18,2018.0,SENTENCIA,60.0,1.0
1,1.0,1.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2018,Ordinaria,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,abr-18,2018.0,1.0,1.0,1.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,2018-06-01,jun-18,2018.0,SENTENCIA,60.0,1.0
2,2.0,2.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,101-2018,Ordinaria,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,abr-18,2018.0,1.0,2.0,2.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,2018-08-29,ago-18,2018.0,SENTENCIA,149.0,1.0
3,3.0,3.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,10-2018,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,ene-18,2018.0,1.0,3.0,3.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,2018-03-14,mar-18,2018.0,SENTENCIA,63.0,1.0
4,4.0,4.0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,102-2018,Ordinaria,812,RECEPTACION. ART. 456 BIS A.,2018-04-02,abr-18,2018.0,1.0,4.0,4.0,10.0,C.A. DE ARICA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,Ordinaria,RECEPTACION. ART. 456 BIS A.,2018-04-02,2018-05-29,may-18,2018.0,SENTENCIA,57.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249506,NaN,NaN,NaN,NaN,1323,NaN,247-2015,NaN,303,NaN,NaT,NaN,NaN,NaN,3605515.0,721435.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,FALSIFICACION O USO MALICIOSO DE DOCUMENTOS PR...,2015-12-09,2016-03-24,mar-16,2016.0,SENTENCIA,106.0,1.0
4249507,NaN,NaN,NaN,NaN,1323,NaN,251-2015,NaN,7037,NaN,NaT,NaN,NaN,NaN,3605526.0,721446.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-12-10,2016-01-30,ene-16,2016.0,SENTENCIA,51.0,1.0
4249508,NaN,NaN,NaN,NaN,1323,NaN,251-2015,NaN,10009,NaN,NaT,NaN,NaN,NaN,3605528.0,721448.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,POSESION O TENENCIA DE ARMAS PROHIBIDAS,2015-12-10,2016-01-30,ene-16,2016.0,SENTENCIA,51.0,1.0
4249509,NaN,NaN,NaN,NaN,1323,NaN,251-2015,NaN,10099,NaN,NaT,NaN,NaN,NaN,3605529.0,721449.0,91.0,C.A. DE SAN MIGUEL,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,Ordinaria,OTROS DELITOS DE LA LEY DE CONTROL DE ARMAS (L...,2015-12-10,2016-01-30,ene-16,2016.0,SENTENCIA,51.0,1.0


In [9]:
df_fulldata_materia.columns

Index(['level_0_x', 'index_x', 'COD. CORTE_x', 'CORTE_x', 'COD. TRIBUNAL',
       'TRIBUNAL_x', 'RIT', 'TIPO CAUSA_x', 'COD. MATERIA', 'MATERIA_x',
       'FECHA INGRESO_x', 'MES INGRESO', 'AÑO INGRESO',
       'TOTAL INGRESOS POR MATERIAS', 'level_0_y', 'index_y', 'COD. CORTE_y',
       'CORTE_y', 'TRIBUNAL_y', 'TIPO CAUSA_y', 'MATERIA_y', 'FECHA INGRESO_y',
       'FECHA TERMINO', 'MES TERMINO', 'AÑO TERMINO', 'MOTIVO TERMINO',
       'DURACION CAUSA', 'TOTAL TERMINOS'],
      dtype='object')

In [10]:
columnas_drop = ['level_0_x', 'index_x', 'level_0_y', 'index_y', 'MES INGRESO', 'MES TERMINO']

df_fulldata_materia.drop(columnas_drop, axis = 'columns', inplace = True)

Aca podemos completar algunos datos faltantes utilizando los datos del dataset INGRESOS:

    - FECHA INGRESO_x en Termino
    - MATERIA_x
o

    - FECHA INGRESO_y en Ingreso
    - MATERIA_y
    
Esto debido a que los datos faltantes se deben principalmente a formas de tramitar las causas en los diferentes tribunales. Esto despues de analizar una cantidad importante de Causas del Tribunal Oral en lo penal de Punta Arenas , cod: Tribunal 991 y junto a Maritza Romero, Jefa de Causas y abogada donde se concluyo:

"Bien. Llevo 5 causas revisadas y sin perjuicio que desentrañaré las otras (3), puedo decir a este altura, que todos los delitos (materias) que no tienen término, obedecen a situaciones a las que el siagj no da respuesta, y dependiendo de los criterios de registro de la tramitación, dan origen a esta falta de término.
 52-2018 , uno de los delitos de la acusación se subsumió (incluyó, por decirlo de algún modo) en el otro, por tanto, el delito subsumido no llevó pena. En esa época, el criterio era dar cuenta de todos los delitos de la acusación, y registrar qué pasó con ellos, y para este caso no había cómo registrar y queda sin término. Actualmente simplemente los eliminamos y ya no damos cuenta de los delitos de la acusación versus sentencia, sino solo acusación.
 111-2017 No hay término para un  delito respecto del cual el tribunal se declaró incompetente.
 155-2017 , caso parecido al primero en el efecto, aunque no en lo jurídico. Acá hubo un concurso aparente de leyes penales, es decir una hecho que puede encuadrarse en dos delitos, y se acusó por los dos , se resolvió prefiriendo uno de ellos, el otro queda sin término."

In [12]:
df_fulldata_materia = df_fulldata_materia.progress_apply(transformdata.faltantes_materia, axis=1)

100%|██████████| 4249511/4249511 [20:28<00:00, 3460.27it/s] 


In [13]:
columnas_drop = ['TIPO CAUSA_y', 'MATERIA_y', 'TRIBUNAL_y', 'COD. CORTE_y', 'CORTE_y', 'FECHA INGRESO_y']

df_fulldata_materia.drop(columnas_drop, axis = 'columns', inplace = True)

In [14]:
df_fulldata_materia.rename(columns = {'COD. CORTE_x':'COD. CORTE',
                                      'CORTE_x':'CORTE',
                                      'TRIBUNAL_x':'TRIBUNAL',
                                      'TIPO CAUSA_x':'TIPO CAUSA',
                                      'MATERIA_x':'MATERIA',
                                      'FECHA INGRESO_x':'FECHA INGRESO'
                                     }, inplace = True)

In [15]:
df_fulldata_materia

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,FECHA TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2018,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,2018.0,1.0,2018-06-01,2018.0,SENTENCIA,60.0,1.0
1,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2018,Ordinaria,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,2018.0,1.0,2018-06-01,2018.0,SENTENCIA,60.0,1.0
2,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,101-2018,Ordinaria,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,2018.0,1.0,2018-08-29,2018.0,SENTENCIA,149.0,1.0
3,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,10-2018,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,2018.0,1.0,2018-03-14,2018.0,SENTENCIA,63.0,1.0
4,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,102-2018,Ordinaria,812,RECEPTACION. ART. 456 BIS A.,2018-04-02,2018.0,1.0,2018-05-29,2018.0,SENTENCIA,57.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249506,91.0,C.A. DE SAN MIGUEL,1323,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,247-2015,Ordinaria,303,FALSIFICACION O USO MALICIOSO DE DOCUMENTOS PR...,2015-12-09,2015.0,NaN,2016-03-24,2016.0,SENTENCIA,106.0,1.0
4249507,91.0,C.A. DE SAN MIGUEL,1323,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,251-2015,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-12-10,2015.0,NaN,2016-01-30,2016.0,SENTENCIA,51.0,1.0
4249508,91.0,C.A. DE SAN MIGUEL,1323,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,251-2015,Ordinaria,10009,POSESION O TENENCIA DE ARMAS PROHIBIDAS,2015-12-10,2015.0,NaN,2016-01-30,2016.0,SENTENCIA,51.0,1.0
4249509,91.0,C.A. DE SAN MIGUEL,1323,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,251-2015,Ordinaria,10099,OTROS DELITOS DE LA LEY DE CONTROL DE ARMAS (L...,2015-12-10,2015.0,NaN,2016-01-30,2016.0,SENTENCIA,51.0,1.0


In [19]:
print(f"Existen un total de {len(df_fulldata_materia)} causas por materias")

Existen un total de 4249511 causas por materias


De los cuales :

In [20]:
df_fulldata_materia['MOTIVO TERMINO'].value_counts()

nan                                                 1104663
DECLARA SOBRESEIMIENTO DEFINITIVO                    755475
APROBACION NO INICIO INVESTIGACION                   639819
SENTENCIA                                            593602
COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD     367823
ACOGE REQUERIMIENTO (MONITORIO)                      318037
NO PERSEVERAR EN EL PROCEDIMIENTO                    211731
DECLARA INCOMPETENCIA                                 99988
CERTIFICA CUMPLIMIENTO ART 468                        65010
ACUMULACION                                           54014
DECLARA INCOMPETENCIA RPA                             23471
DECLARA INADMISIBILIDAD DE LA QUERELLA                 9906
ABANDONO DE LA QUERELLA                                5972
Name: MOTIVO TERMINO, dtype: int64

Acá analizamos si existe alguna relación entre Total Ingresos por Matería y Total Terminos.

In [21]:
df_fulldata_materia[df_fulldata_materia['TOTAL INGRESOS POR MATERIAS']!=df_fulldata_materia['TOTAL TERMINOS']]

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,FECHA TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
21,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,113-2018,Ordinaria,14004,CONDUCCION EBRIEDAD SUSP.LIC. ART.196Y209 INC....,2018-04-11,2018.0,1.0,NaT,NaN,nan,NaN,NaN
30,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,120-2018,Ordinaria,702,HOMICIDIO.,2018-04-16,2018.0,2.0,2018-06-30,2018.0,SENTENCIA,75.0,1.0
40,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,128-2018,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-19,2018.0,2.0,2018-06-11,2018.0,SENTENCIA,53.0,1.0
47,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,134-2018,Ordinaria,7038,CONSUMO/PORTE EN LUG.PUB.O PRIV.C/PREVIO CONCI...,2018-04-19,2018.0,1.0,NaT,NaN,nan,NaN,NaN
48,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,134-2018,Ordinaria,14004,CONDUCCION EBRIEDAD SUSP.LIC. ART.196Y209 INC....,2018-04-19,2018.0,1.0,NaT,NaN,nan,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249506,91.0,C.A. DE SAN MIGUEL,1323,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,247-2015,Ordinaria,303,FALSIFICACION O USO MALICIOSO DE DOCUMENTOS PR...,2015-12-09,2015.0,NaN,2016-03-24,2016.0,SENTENCIA,106.0,1.0
4249507,91.0,C.A. DE SAN MIGUEL,1323,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,251-2015,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-12-10,2015.0,NaN,2016-01-30,2016.0,SENTENCIA,51.0,1.0
4249508,91.0,C.A. DE SAN MIGUEL,1323,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,251-2015,Ordinaria,10009,POSESION O TENENCIA DE ARMAS PROHIBIDAS,2015-12-10,2015.0,NaN,2016-01-30,2016.0,SENTENCIA,51.0,1.0
4249509,91.0,C.A. DE SAN MIGUEL,1323,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,251-2015,Ordinaria,10099,OTROS DELITOS DE LA LEY DE CONTROL DE ARMAS (L...,2015-12-10,2015.0,NaN,2016-01-30,2016.0,SENTENCIA,51.0,1.0


Vemos que existen cerca de 30 mil ingresos por materia donde se diferencian... ¿cual es el motivo?

- Se puede deber a que pueden ser más de un imputado, donde se termino para uno de ellos.

-----------------------------------------------------------------

In [22]:
filtro_oral = df_fulldata_materia[df_fulldata_materia['TRIBUNAL'].str.contains('ORAL')]
filtro_garantia = df_fulldata_materia[df_fulldata_materia['TRIBUNAL'].str.contains('GARANTIA')]

In [23]:
filtro_oral

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,FECHA TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
0,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2018,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-04-02,2018.0,1.0,2018-06-01,2018.0,SENTENCIA,60.0,1.0
1,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2018,Ordinaria,10004,"PORTE ILEGAL DE ARMA DE FUEGO, MUNICIONES Y OT...",2018-04-02,2018.0,1.0,2018-06-01,2018.0,SENTENCIA,60.0,1.0
2,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,101-2018,Ordinaria,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2018-04-02,2018.0,1.0,2018-08-29,2018.0,SENTENCIA,149.0,1.0
3,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,10-2018,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2018-01-10,2018.0,1.0,2018-03-14,2018.0,SENTENCIA,63.0,1.0
4,10.0,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,102-2018,Ordinaria,812,RECEPTACION. ART. 456 BIS A.,2018-04-02,2018.0,1.0,2018-05-29,2018.0,SENTENCIA,57.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249506,91.0,C.A. DE SAN MIGUEL,1323,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,247-2015,Ordinaria,303,FALSIFICACION O USO MALICIOSO DE DOCUMENTOS PR...,2015-12-09,2015.0,NaN,2016-03-24,2016.0,SENTENCIA,106.0,1.0
4249507,91.0,C.A. DE SAN MIGUEL,1323,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,251-2015,Ordinaria,7037,TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).,2015-12-10,2015.0,NaN,2016-01-30,2016.0,SENTENCIA,51.0,1.0
4249508,91.0,C.A. DE SAN MIGUEL,1323,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,251-2015,Ordinaria,10009,POSESION O TENENCIA DE ARMAS PROHIBIDAS,2015-12-10,2015.0,NaN,2016-01-30,2016.0,SENTENCIA,51.0,1.0
4249509,91.0,C.A. DE SAN MIGUEL,1323,TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO,251-2015,Ordinaria,10099,OTROS DELITOS DE LA LEY DE CONTROL DE ARMAS (L...,2015-12-10,2015.0,NaN,2016-01-30,2016.0,SENTENCIA,51.0,1.0


In [24]:
filtro_garantia

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,COD. MATERIA,MATERIA,FECHA INGRESO,AÑO INGRESO,TOTAL INGRESOS POR MATERIAS,FECHA TERMINO,AÑO TERMINO,MOTIVO TERMINO,DURACION CAUSA,TOTAL TERMINOS
615,10.0,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,10000-2018,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-12-06,2018.0,1.0,NaT,NaN,nan,NaN,NaN
616,10.0,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,10001-2018,Ordinaria,709,LESIONES GRAVES.,2018-12-06,2018.0,1.0,NaT,NaN,nan,NaN,NaN
617,10.0,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,10001-2018,Ordinaria,7007,TRAFICO ILICITO DE DROGAS (ART. 3).,2018-12-06,2018.0,1.0,NaT,NaN,nan,NaN,NaN
618,10.0,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,1000-2018,Ordinaria,848,HURTO SIMPLE POR UN VALOR DE MEDIA A MENOS DE ...,2018-02-01,2018.0,1.0,NaT,NaN,nan,NaN,NaN
619,10.0,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,1000-2018,Ordinaria,13028,HURTO FALTA (494 BIS CODIGO PENAL).,2018-02-01,2018.0,1.0,2018-02-08,2018.0,COMUNICA Y/O APLICA DECISION PPIO DE OPORTUNIDAD,7.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4249456,91.0,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA TALAGANTE,888-2016,Ordinaria,812,RECEPTACION. ART. 456 BIS A.,2016-02-11,2016.0,NaN,2016-01-29,2016.0,SENTENCIA,13.0,1.0
4249457,91.0,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA TALAGANTE,888-2016,Ordinaria,812,RECEPTACION. ART. 456 BIS A.,2016-02-11,2016.0,NaN,2016-02-09,2016.0,DECLARA INCOMPETENCIA,2.0,1.0
4249458,91.0,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA TALAGANTE,901-2015,Ordinaria,303,FALSIFICACION O USO MALICIOSO DE DOCUMENTOS PR...,2015-02-20,2015.0,NaN,2016-06-09,2016.0,SENTENCIA,475.0,1.0
4249459,91.0,C.A. DE SAN MIGUEL,1240,JUZGADO DE GARANTIA TALAGANTE,93-2016,Ordinaria,12149,DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIV...,2016-01-07,2016.0,NaN,2016-01-04,2016.0,DECLARA INCOMPETENCIA,3.0,1.0


In [25]:
filtro_oral['TRIBUNAL'].value_counts()

TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE             6225
SEXTO TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO      5992
SEGUNDO TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO    5593
TRIBUNAL DE JUICIO ORAL EN LO PENAL RANCAGUA            5140
CUARTO TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO     5130
TRIBUNAL DE JUICIO ORAL EN LO PENAL VALPARAISO          4839
TRIBUNAL DE JUICIO ORAL EN LO PENAL VIÑA DEL MAR        4570
TRIBUNAL DE JUICIO ORAL EN LO PENAL CONCEPCION          4441
SEPTIMO TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO    3862
TRIBUNAL DE JUICIO ORAL EN LO PENAL SAN BERNARDO        3722
TRIBUNAL DE JUICIO ORAL EN LO PENAL ANTOFAGASTA         3658
TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA               3655
TERCER TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO     3400
TRIBUNAL DE JUICIO ORAL EN LO PENAL TALCA               2913
TRIBUNAL DE JUICIO ORAL EN LO PENAL LA SERENA           2556
PRIMER TRIBUNAL DE JUICIO ORAL EN LO PENAL SANTIAGO     2547
TRIBUNAL DE JUICIO ORAL 

In [26]:
filtro_oral['MATERIA'].value_counts()

TRAFICO DE PEQUEÑAS CANTIDADES (ART. 4).                        9655
TRAFICO ILICITO DE DROGAS (ART. 3).                             6882
RECEPTACION. ART. 456 BIS A.                                    6819
ROBO CON INTIMIDACION.                                          6395
DESACATO (ART. 240 CODIGO DE PROCEDIMIENTO CIVIL).              5246
                                                                ... 
OTROS DELIT QUE AFECTAN LOS DCHOS GARANT. POR LA CONSTITUC         1
CAZA Y COMERCIALIZACION DE ESPECIES PROHIBIDAS (ART. 31 LEY.       1
INFRACCION L.O.C DEL BANCO CENTRAL.                                1
AMPLIACION DE PARTE.                                               1
VIOLACION DE MENOR DE 14 AÑOS                                      1
Name: MATERIA, Length: 359, dtype: int64

In [27]:
filtro_oral['MOTIVO TERMINO'].value_counts()

SENTENCIA                            83230
nan                                  25260
DECLARA SOBRESEIMIENTO DEFINITIVO     2085
ACUMULACION                            783
DECLARA INCOMPETENCIA                  249
ABANDONO DE LA QUERELLA                 54
DECLARA INCOMPETENCIA RPA                3
Name: MOTIVO TERMINO, dtype: int64

## CARGA DATA A ARCHIVOS FUTHER

In [47]:
# Directorio donde se guardaran archivos feather
df_fulldata_materia.reset_index(inplace = True)
filtro_oral.reset_index(inplace = True)
filtro_garantia.reset_index(inplace = True)
os.makedirs(path_processed, exist_ok = True)  

# Guardamos dataset como archivo feather

df_fulldata_materia.to_feather(f'{path_processed}/Consolidado_Materia.feather')
filtro_oral.to_feather(f'{path_processed}/JuicioOralesMateria.feather')
filtro_garantia.to_feather(f'{path_processed}/CausasGarantiaMateria.feather')